<center> <h1>Ecoli Tracking Script</h1> </center>
Written by Michael Isselstein

Please insert the folder containing the "Tracking_functions.py" file. This file contains routines for the evaluation that need to be loaded before the program can execute
__<span style="color: red;">Change me!</span>__

In [ ]:
path_functions = r"C:\Users\"

First, Necessary libraries are imported. So __do not change the following cell__

In [ ]:
import os
cwd = os.getcwd()
os.chdir(path_functions)
import Tracking_functions as fcts
os.chdir(cwd)
import numpy as np
import pandas as pd


import pims


import trackpy as tp

import time

import matplotlib.pyplot as plt 
import imageio as io
import skimage
from skimage.morphology import disk
from skimage.filters import median
import skimage.measure as measure
import skimage.morphology as morphology

import ipywidgets as widgets

%matplotlib qt

## Insert location of data and location where results should be stored. <br>The script will go through every file ending with ".tif"
__<span style="color: red;">Change me!</span>__

In [ ]:
raw_data_path = r"C:\Users\"
results_path = r"C:\Users\"

## Now, cells will be detected. For this, answer whether movies are pre-processed binary images or raw images. <br><br>For raw images, an automated image processing routine will be applied to the movie. <br><br>__This routine and their parameters are based on initial screenings and should not be used blindly, especially for different imaging conditions__
__<span style="color: red;">Give input upon execution!</span>__

In [ ]:
preprocess = input("Are cells already defined in binary image (y/n)? Type \"n\" if automated pre-processing is necessary")
movie_names = [n]
cell_loc = [n]
for file in os.listdir(raw_data_path):
    if file.endswith(".tif"):
        movie_names.append(file)
        cell_loc.append(fcts.cell_detection(raw_data_path, file, preprocess))
cell_loc = pd.concat(cell_loc)
print("The following movies were used: ")
for i in range(len(movie_names)):
    print('\x1b[1;31m', i, ": ", movie_names[i], '\x1b[0m')

<center> <h1>Visualize detection accuracy / movement of cells. <br><br>&rarr; Display tracked cell position compared to a different image</h1> </center>

Select the displayed image as __"image_frame"__ and the frame to show the tracked movement from as __"detect_frame"__

__<span style="color: red;">Change me!</span>__

In [ ]:
test_movie_index = 1
image_frame = 5
detect_frame = 10

search_range = 70
memory = 1

In [ ]:
raw_img = pims.open(os.path.join(raw_data_path, movie_names[test_movie_index]))
cells = cell_loc[cell_loc["Movie"] == movie_names[test_movie_index]]
tracks = tp.link(cells, search_range, memory = memory);
tp.annotate(cells[cells.frame == (detect_frame)], raw_img[image_frame]);

## Try to link detected cells to cells in subsequent frmaes (tracking)
__"search_range" and "memory" are parameters for the tracking. This section is intended to visualize the behavior of those parameters. Choose optimal parameters and use them for the final evaluation__

Explanation on the tracking parameters can be found under http://soft-matter.github.io/trackpy/dev/generated/trackpy.link.html

In [ ]:
from ipywidgets import interact, interactive, IntSlider, Layout, fixed, Checkbox, FloatSlider
fig, (ax1, ax2) = plt.subplots(1, 2)



def check_track(cells, frame, raw_img, displacement,  memory, search_range, contrast, arrow):
    tracks = tp.link(cells, search_range, memory = memory)
    t0 = tracks[tracks.frame == frame]
    t1 = tracks[tracks.frame == frame + displacement]
    ax1.cla()
    ax2.cla()
    # tp.annotate(t0, raw_img[frame], ax = ax1, imshow_style = {"cmap": "gray_r", "vmin": 1.5*np.amin(raw_img[frame])})
    tp.annotate(t1, raw_img[frame + displacement], ax = ax2, imshow_style = {"vmin": contrast*np.amin(raw_img[frame + displacement])})
    tp.annotate(t0, raw_img[frame], ax = ax1, imshow_style = {"vmin": contrast*np.amin(raw_img[frame])})
    # tp.annotate(t1, raw_img[frame + displacement], ax = ax2, imshow_style = {"cmap": "gray_r", "vmin": 1.5*np.amin(raw_img[frame + displacement])})
    # ax2.imshow(raw_img[frame +1])
    # ax1.imshow(raw_img[frame], cmap = "gray")
    # ax2.imshow(raw_img[frame +1], cmap = "gray")
    # ax1.scatter(t0.x, t0.y, facecolors = "None")
    # ax2.scatter(t1.x, t1.y, facecolors = "None")
    if arrow:
        tp.plot_displacements(tracks, frame + displacement, frame, ax = ax2)
        ax2.invert_yaxis()
    ax1.set_title("Frame {}".format(frame))
    ax2.set_title("Frame {}".format(frame + displacement))

    
interactive_plot = interactive(check_track, 
                               cells = fixed(cells),
                               frame = IntSlider(value = 0, min = 0, max = len(raw_img) - 1, step = 1),
                               displacement = IntSlider(value = 3, min = 0, max = len(raw_img)-1, step = 1),
                               raw_img = fixed(raw_img),
                               search_range = IntSlider(value = 70, min = 0, max = 100, step = 1),
                               memory = IntSlider(value = 3, min = 0, max = 20, step = 1),                               
                               contrast = FloatSlider(value = 1.4, min = 1, max = 3, step = 0.1, description = "Display Contrast-Factor"),
                               arrow = Checkbox(value=True, description='Arrows', disabled=False, indent=False))
interactive_plot

## Display Detected trajectories with interactive plot
The variable __search_range__ defines the area in which the tracking algorithm searches for cells in subsequent images

The variable __memory__ defines how long the algorithm should remember a cell once it has left the FOV (in case it re-appears in subsequent images)

__<span style="color: red;">Change me!</span>__

In [ ]:
search_range = 70
memory = 3

In [ ]:
from ipywidgets import interact, interactive, IntSlider, Layout, fixed
fig, ax1 = plt.subplots(1, 1)
tracks = tp.link(cells, search_range, memory = memory)
for track in set(tracks.particle):
    ax1.plot(tracks[tracks.particle == track].sort_values("frame").x, tracks[tracks.particle == track].sort_values("frame").y, alpha=0.5)
image = ax1.imshow(raw_img[0], cmap="gray")





# movie = np.asarray([raw_img[i] for i in range(len(raw_img))])
def check_track(frame, raw_img, contrast):
    image.set_data(raw_img[frame])
    image.set(clim = (contrast * np.amin(raw_img[frame]), np.amax(raw_img[frame])))
    ax1.set_title("Frame {}".format(frame))

    
interactive_plot = interactive(check_track, 
                               frame = IntSlider(value = 0, min = 0, max = len(raw_img) - 2, step = 1),
                               raw_img = fixed(raw_img),
                               contrast = FloatSlider(value = 1.4, min = 1, max = 3, step = 0.1, description = "Display Contrast-Factor"))
                               

interactive_plot

<center> <h1>Export Data</h1> </center>


__<span style="color: green;">First</span>, insert optimal tracking parameters and display contrast__

__<span style="color: green;">Secondly</span>, define Final Movie resolution in *dpi*, aswell as framerate in framer per second (*fps*).__

__<span style="color: green;">Finally</span>, define parameters necessary for data evaluation. The parameter *lower_angle_threshold* is the angle (in °) under which directional change is considered low__

__*Magnification* is the magnification of the microscope, *px_size* is the edge-length of a physical Camera Pixel in µm (multiply with binning if pixels are binned)__

__*fps* are the frames per second of the recorded movie. *min_track_length* defines the minimal track-length for tracks to be saved in one of the aggregated files. There will still be a separate file contaiing all tracks__

__<span style="color: red;">Change me!</span>__

In [ ]:
#Tracking Parameters for exported data:
search_range = 70
memory = 1
Display_Contrast = 1.4

#Export movie parameters
dpi = 300 #dpi of the exported "tracking movie"
output_fps = 10 #fps of the exported "tracking movie"

#Evaluation criteria
lower_angle_threshold = 20
magnification = 40 #total magnification of optical system
px_size = 6.45 #physical pixel edge-length in µm
fps = 10 #fps of recorded movie
min_track_length =  5 #Tracks shorter than min_track_lenght will be excluded in one of the results-files. There will still be a separate file containing all tracks

In [ ]:
#Batch extract tracks from all files
cells_tracked = []
for file in range(len(set(cell_loc["Movie"]))):
    cells_tracked.append(tp.link(cell_loc[cell_loc["Movie"] == movie_names[file]], search_range, memory = memory))
cells_tracked = pd.concat(cells_tracked)
results = fcts.evaluate_tracks(cells_tracked, magnification, px_size, fps, lower_angle_threshold)
results = results.sort_values(["Movie", "particle"])
cells_tracked = cells_tracked.sort_values(["Movie", "particle", "frame"])

## Create the tracking movie and export the aggregated Data

In [ ]:
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

def find_data(path, filename):
    folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(folder,path)) and (filename in folder)]
    return folders

def find_latest_iteration(path, filename):
    folders = find_data(path, filename)
    iterations = [folder.split("_")[-1] for folder in folders]
    iterations = [int(iteration) for iteration in iterations if iteration.isnumeric()]
    if len(iterations)>0:
        return max(iterations)
    else:
        return 0

def new_data_folder(path, filename):
    last_it = find_latest_iteration(path, filename)
    new_foldername = os.path.join(path, filename + "_" + str(int(last_it) + 1))
    return new_foldername

#Explanation from matplotlib and https://stackoverflow.com/questions/4092927/generating-movie-from-python-without-saving-individual-frames-to-files
def ani_frame(tracks, raw_img):
    fig, ax = plt.subplots(1, 1)
    ax.get_yaxis().set_visible(False)
    image = ax.imshow(raw_img[0], cmap="gray")
    lines = ax.plot(np.empty((np.amax(tracks.frame), np.amax(tracks.particle)+1)), np.empty(( np.amax(tracks.frame), np.amax(tracks.particle)+1)), alpha=0.5)
    def update_img(n):
        ax.set_title("Frame {}".format(n))
        _ = image.set_data(raw_img[n])
        for track in set(tracks.particle):
            part = tracks[tracks.particle == track].sort_values("frame")
            lines[track].set_data(part.x[part.frame < n], part.y[part.frame < n])
        return lines
    anim = FuncAnimation(fig, update_img, frames = len(raw_img), interval = 10, repeat=True)
    return anim

results_folder = new_data_folder(results_path, raw_data_path.split("\\")[-1])
os.mkdir(results_folder)
for movie in set(cell_loc["Movie"]):
    cells = cell_loc[cell_loc["Movie"] == movie]
    tracks = cells_tracked[cells_tracked["Movie"] == movie]
    '''
    Create animation and destination folder and save animation in destination folder
    '''
    tracks = tp.link(cells, search_range, memory = memory)
    anim = ani_frame(tracks, raw_img);
    
    destination = os.path.join(results_folder, movie+ ".mp4")
    writer = animation.writers["ffmpeg"](fps=fps)
    anim.save(destination, dpi=dpi, writer = writer)
    plt.show()

'''
Aggregate Data and export
'''
data = fcts.evaluate_tracks(tracks_mult_mov = cells_tracked, magnification = magnification, px_size = px_size,\
                            fps = fps, low_angle = lower_angle_threshold)
data.to_csv(os.path.join(results_folder, "FullData.tsv"), sep="\t")
cells_tracked.to_csv(os.path.join(results_folder, "Tracks_with_searchrange={}_memory={}.tsv".format(search_range, memory)), sep="\t")
data_thresh = data[data["number_of_frames"] >= min_track_length]
data_thresh.to_csv(os.path.join(results_folder, "Data_thresh = {}.tsv".format(min_track_length)), sep="\t")
print("Done")

<center> <h1>Description of exported parameters</h1> </center>

| Parameter      | Description |
| ----------- | ----------- |
| __Movie__      | Filename of movie for respective particle       |
| __particle__   | Index of particle        |
| __distance[µm]__   | Calculated total distance traveled over duration of movie in µm        |
| __track_duration [frames]__   | duration of track in unit frames        |
| __number_of_frames [frames]__   | Number of frames that the particle was detected        |
| __track_duration [s]__   | duration of track in unit seconds        |
| __mean_stepsize[µm]__   | Average of linear distance between positions in µm        |
| __std_stepsize[µm]__   | Standard deviation of linear distance between positions in µm        |
| __mean_velocity[µm]__   | Average of linear distance between positions per second in µm/s        |
| __std_velocity [µm/s]__   | Standard deviation of linear distance between positions per second in µm/s        |
| __rel_cw_rot__   | Fraction of steps going in a clockwise direction        |
| __rel_ccw_rot__   | Fraction of steps going in a counterclockwise direction        |
| __rel_low_dir_ch__   | Fraction of steps going in angles smaller than defined "low_angle" variable        |
| __av_angle [°]__   | Average of the angle in Degrees        |
| __std_angle [°]__   | Standard deviation of the angle in Degrees        |
| __mean_velocity / distance [1/s]__   | Mean Velocity divided by the total distance. Unit is 1/s        |
| __mean_stepsize / distance__   | Mean stepsize divided by the total distance. Per definition unitles        |
| __std_position [µm]__   | Standard Deviation of cell position. Calculated as sqrt(var(x) + var(y)), namely the squareroot of the sum of the positional variances        |